# Table of Contents

1. [Header](#header)
2. [Particles](#particles)
3. [FFT Data](#fft)
4. [Halo and Subhalo](#halo)<br>
    4.1. [FOF Data](#fof)<br>
    4.2. [SUBFIND Catalog](#subcat)

### This notebook gives examples of how to use read_indra.py to read in Indra data on the SciServer.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import indratools as indra

Initialize the `runid` that specifies the Indra run and the snapshot of the particle data (`snapnum`) and FFT data (`tnum`) for redshift 0. `runid` can either be an integer from 0 to 511 or a tuple of (`X`,`Y`,`Z`) integers (each from 0 to 7), corresponding to the raveled and unraveled indices of an 8x8x8 cube.

In [ ]:
runid = (2, 0, 0); snapnum = 63; tnum = 504

You can convert between raveled and unraveled Indra run IDs using the `Run` class.

In [ ]:
run = indra.Run(runid)
print(run.num,run.X,run.Y,run.Z)

The directory location `datadir` of run X_Y_Z on the FileDB is determined by `get_loc` and is the default data location. If `datascope = True` this sets `datadir` to the datascope directory location, e.g. `/datascope/indra4/4_5_6`. When `datadir` is specified in the reading functions, it determines which simulation is read, and `runid` is ignored.

In [ ]:
indra.get_loc(runid)

# Header
<a id="header"></a>

Length data are in units of Mpc/h, velocities in km/s, and masses in MSun/h. Set `verbose = True` to print the `datadir`.

In [ ]:
header = indra.getheader(runid,snapnum,verbose=True)
print(header.keys())

In [ ]:
print(header['num_files'],header['npart'],header['mass'],header['time'],header['BoxSize'])

# Particles
<a id="particles"></a>

Read the 64 snapshot numbers, redshifts, and scalefactors (for particle and halo data).

In [ ]:
snaps,redshifts,times = indra.part_snapinfo()

Or retrieve redshifts and scalefactors for a subset of snapnumbers.

In [ ]:
thisz, thisa = indra.part_snapinfo([41,snapnum])
print(thisz,thisa)

Read positions only, no sorting:

In [ ]:
pos_ns = indra.getpos(runid,snapnum)
print(pos_ns[0])

Get velocities too, and sort particles and velocities by their IDs:

In [ ]:
pos,vel,ids = indra.getparticles(runid,snapnum,sort=True)

In [ ]:
print(pos[0],vel[0],np.max(ids))

# FFT Data
<a id="fft"></a>

Read the 505 snapshot numbers, redshifts, and scalefactors for FFT data (or a subset as with the particle snapshots).

In [ ]:
fft_snaps,fft_redshifts,fft_times = indra.FFT_snapinfo()

Compute the k vectors. These arrays have the same shape as the FFT data.

In [ ]:
kx,ky,kz = indra.getkvals()
k = np.sqrt(kx*kx+ky*ky+kz*kz)

Before reading, define some functions to calculate the power spectrum of this gridded FFT data.

In [ ]:
def powspec(fft_re,fft_im,nbins=100, k=k):
    ps = fft_re*fft_re+fft_im*fft_im
    ps = ps[k>0] # remove k = 0
    k = k[k>0]
    
    # average PS in logarithmic bins of k
    ps1d, kbin = np.histogram(np.log10(k),nbins,weights=ps)
    counts = np.histogram(np.log10(k),nbins)[0]
    ps1d = ps1d[counts>0]/counts[counts>0] 
    
    binvals = kbin[0:nbins]+np.diff(kbin)//2
    binvals = binvals[counts>0]
    poisson = 1/np.sqrt(counts[counts>0])
    
    pk = {'k':10**binvals, 'ps': ps1d, 'poisson': poisson}
    return pk

In [ ]:
# the cosmology is hard-coded here but om = header['omega0']
def growthfunc(a, om=0.272):
    ol = 1-om
#    a = 1./(1.+z)
    da=a/10000.
    integral = 0.
    for i in range(10000):
        aa=(i+1)*da
        integral+=da/((aa*np.sqrt(om/(aa**3)+ol))**3)

    return 5*om/2*np.sqrt(om/a**3+ol)*integral

Read the CAMB linear power spectrum, which is normalized to z=0. (So to compare to Plin, multiply PS(z) by D(z)^2/D(z=0)^2.)

In [ ]:
kth,pth = indra.get_pklin()

Now plot the ratio of power spectra from the FFT data to the linear theory PS at different redshifts.

In [ ]:
fft_re,fft_im,a = indra.getfft(runid,tnum)
pk = powspec(fft_re,fft_im) # need bins to interpolate linear PS
bins = pk['k']
plin = np.interp(bins,kth,pth)
norm = header['BoxSize']**3/header['npart']**2
plt.figure(figsize=(8,6))
for i in np.arange(0,501,100):
    fft_re,fft_im,a = indra.getfft(runid,i)
    pk = powspec(fft_re,fft_im)
    normL = growthfunc(a)**2/growthfunc(1)**2
    plt.plot(pk['k'],(pk['ps']*norm)/(plin*normL),label="{:.3f}".format(a)) 
plt.xscale('log')
plt.legend(loc='upper left')
plt.xlabel(r'$k$ (h/Mpc)',size='large')
plt.ylabel(r'$P(a,k)/P_{lin}(k)$',size='large');

Now measure the mean and variance of P(k) at redshift 1, excluding the first 128 simulations (which might be re-run in the future; see the paper for explanation.)

In [ ]:
tnum = 327
runstart = 128
runfinish = 448
numruns = runfinish-runstart
# first read one to get size of bins
fft_re,fft_im,a = indra.getfft(runid,tnum)
pk = powspec(fft_re,fft_im)
bins = pk['k']
psz1 = np.zeros([len(bins),numruns])
norm = header['BoxSize']**3/header['npart']**2
for i,s in enumerate(range(runstart,runfinish)):
    fft_re,fft_im,a = indra.getfft(s,tnum)
    pk = powspec(fft_re,fft_im)
    psz1[:,i] = pk['ps']*norm
psmean = np.mean(psz1,1)
psvar = np.std(psz1,1)

In [ ]:
plt.figure(figsize=(8,6))
plt.loglog(bins,psmean)
plt.fill_between(bins,psmean-psvar,psmean+psvar,alpha=.3)
plt.xlabel(r'$k$ (h/Mpc)',size='large')
plt.ylabel(r'$P(k)$ ($h^{-1}$Mpc)$^3$',size='large')
plt.title('z = {:.2} Power Spectra from FFT data, {} runs'.format(fft_redshifts[tnum],numruns),size='large');

# Halo and Subhalo
<a id="halo"></a>

The FOF (or 'group') and SUBFIND headers contain the total number of groups/subhalos.

In [ ]:
TotNgroups = indra.getsubheader(runid,snapnum,getfof=True)
print(TotNgroups)
TotNsubs = indra.getsubheader(runid,snapnum)
print(TotNsubs)

## FOF Data
<a id="fof"></a>

If all you want is the number of particles in each FOF halo (to multiply by `header['mass']`, for example), this is given by `groupLen`.

In [ ]:
groupLen = indra.getfof(runid,snapnum)
print(np.min(groupLen),np.max(groupLen))

`groupOffset` gives the information needed to index `groupids`, the particle IDs in each group.

In [ ]:
groupLen,groupOffset,groupids = indra.getfofids(runid,snapnum)

The IDs of the particles in halo i are given by:

In [ ]:
i = 0
haloIDs = groupids[groupOffset[i]:groupOffset[i]+groupLen[i]]

Let's plot some particles! I know that halo 0 (in snapshot 63 of 2_0_0) doesn't cross any boundaries, so we can ignore periodic boundary conditions for now. This will only work if you called `getparticles` with `sort=True`.

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(pos[haloIDs,0],pos[haloIDs,1],'m.')
plt.xlabel('x (Mpc/h)')
plt.ylabel('y (Mpc/h)');

## Subhalo Catalog
<a id="subcat"></a>

There isn't much to the FOF data, since the unbinding procedure and calculation of halo properties is done by SUBFIND.

In [ ]:
cat = indra.getsubcat(runid,snapnum)
subids = indra.getsubids(runid,snapnum)

Some properties are defined for every FOF group, and some for every subhalo.

In [ ]:
for key in cat.keys():
    keytype = 'TotNFOF' if len(cat[key]) == TotNgroups else 'TotNSub'
    print('{} has length {} ({})'.format(key,keytype,len(cat[key])))

In [ ]:
print("mean M200 = {:.3e} Msun/h; mean R200 = {:.3} Mpc/h; mean velocity = {:.4} km/s".format(np.mean(cat['M200crit']),np.mean(cat['R200crit']),np.mean(np.abs(cat['SubVel']))))

Indexing can get a bit tricky, so here are some examples. First let's pick the most massive FOF group with 5 subhalos.

In [ ]:
has5 = np.where(cat['NsubPerHalo'] == 5)[0]
thishalo = np.argmax(cat['M200crit'][has5])
thishalo = has5[thishalo] # get the index of this halo in the full array
print('halo index = {}, M200 = {:.3e} Msun/h, R200 = {:.3} Mpc/h'.format(thishalo,cat['M200crit'][thishalo],cat['R200crit'][thishalo]))

The IDs of the particles in this FOF halo (before unbinding) can be found via:

In [ ]:
haloIDs = groupids[groupOffset[thishalo]:groupOffset[thishalo]+groupLen[thishalo]]

Let's get the indices of the subhalos with this parent halo. We can do this in two ways.

In [ ]:
print(np.where(cat['subParentHalo'] == thishalo)[0])
subindices = cat['FirstSubOfHalo'][thishalo]+np.arange(cat['NsubPerHalo'][thishalo])
print(subindices)

The halo positions are calculated for every subhalo, so let's define the center as that of the main subhalo, and take care of periodic boundary conditions.

In [ ]:
boxsize = header['BoxSize']
center = cat['SubPos'][cat['FirstSubOfHalo'][thishalo],:]
print(center)
halopos = pos[haloIDs,:] - center
halopos[halopos < boxsize/2.] += boxsize
halopos[halopos > boxsize/2.] -= boxsize
print(np.mean(halopos,axis=0))

Now let's plot the particles in these halos! We'll plot the main halo in black first, so any particles that remain black were unbound by SUBFIND and don't belong to any subhalos (including the main `FirstSubOfHalo`).

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(halopos[:,0],halopos[:,1],marker = '.',color='k')
colors = ['g','b','m','r','c']
for c, i in zip(colors,subindices):
    subIDs = subids[cat['subOffset'][i]:cat['subOffset'][i]+cat['subLen'][i]]
    subpos = pos[subIDs,:] - center
    subpos[subpos < boxsize/2.] += boxsize
    subpos[subpos > boxsize/2.] -= boxsize
    plt.scatter(subpos[:,0],subpos[:,1],marker ='.',color=c)
plt.xlabel('x (Mpc/h)',size='large')
plt.ylabel('y (Mpc/h)',size='large');